In [49]:
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import datetime

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [60]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path)
browser = Browser('chrome', **executable_path, headless=False)
# browser = Browser('chrome')

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [/Users/abby/.wdm/drivers/chromedriver/mac64/83.0.4103.39/chromedriver] found in cache


In [61]:
base_url = 'https://www.walmart.com'
search_url = '/search/?query=room%20air%20purifier'
browser.visit(base_url + search_url)

In [62]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [63]:
products = soup.find_all('div', class_='search-result-gridview-item')

In [64]:
productURL_list = []
productImage_list = []
productTitle_list = []
starReview_list = []
reviewAmount_list = []
currentPrice_list = []

reviewURL_list = []


today = datetime.datetime.now()

for product in products:

    link = product.find('a')
    href = link['href']
    product_url = base_url + href
    productURL_list.append(product_url)
#     print(product_url)

    img = product.find('img')['src']
    productImage_list.append(img)
#     print(img)

    product_title = product.find('img')['alt']
    productTitle_list.append(product_title)
#     print(product_title)

    stars_review = product.find('span', class_='visuallyhidden seo-avg-rating').text   
    starReview_list.append(stars_review)
#     print(stars_review)
    
    review_amount = product.find('span', class_='seo-review-count visuallyhidden').text
    reviewAmount_list.append(review_amount)
#     print(f"{review_amount} reviews")
    
    if int(review_amount) > 0:
        review_url = product.find('div', class_='stars').find('a')['href']
#         print(base_url + review_url)
    else:
        review_url = "NaN"
    reviewURL_list.append(review_url)

    price = product.find('span', class_='price-main')
    current_price = price.find('span', class_='visuallyhidden').text
    currentPrice_list.append(current_price)
#     print(current_price)
    
browser.quit()
    
data = {
    "Title": productTitle_list,
    "URL": productURL_list,
    "Image": productImage_list,
    "NumberOfStars": starReview_list,
    "NumberofReviews": reviewAmount_list,
    "ReviewsURL": reviewURL_list,
    "Price": currentPrice_list
       }

product_df = pd.DataFrame.from_dict(data)
product_df.head()

AttributeError: 'NoneType' object has no attribute 'text'

In [47]:
browser.visit(product_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [48]:
soup

<html lang="en"><head>
<meta charset="utf-8"/><meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/><link href="//i5.wal.co" rel="dns-prefetch"/><link href="//schema.org" rel="dns-prefetch"/><link href="//b.wal.co" rel="dns-prefetch"/><link href="//beacon.walmart.com" rel="dns-prefetch"/><link href="//tpc.googlesyndication.com" rel="dns-prefetch"/><link href="//partner.googleadservices.com" rel="dns-prefetch"/><link href="//pagead2.googlesyndication.com" rel="dns-prefetch"/><link href="//p.typekit.net" rel="dns-prefetch"/><link href="//omniture.walmart.com" rel="dns-prefetch"/><link href="//omniture-ssl.walmart.com" rel="dns-prefetch"/><link href="//pubads.g.doubleclick.net" rel="dns-prefetch"/><link href="//beam.hlserve.com" rel="dns-prefetch"/><link href="//secure-assets.rubiconproject.com" rel="dns-prefetch"/><link href="//ib.adnxs.com" rel="dns-prefetch"/><link href="//dis.criteo.com" rel="dns-prefetch"/><meta content="width=device-width, initial-scale=1.0" name="viewport"

In [ ]:
soup.execute_script("window.scrollTo(0, document.body.scrollHeight);")